<a href="https://kaggle.com/kernels/welcome?src=https://github.com/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" width=120 /></a>

<a href="https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width=120 /></a>

In [ ]:
#@title Nombre del estudiante
Estudiante = "Judith Brito" #@param {type:"string"}
Código = "212438" #@param {type:"string"}

# "Datascraping con Twitter"


Uno de los superpoderes de la AI/ML es que nos permite entender el mundo desde la perspectiva de los datos.

Twitter es una plataforma que proporciona datos extensos en lenguaje natural, en base a los cuales se puede explorar mucho.

In [ ]:
# importing libraries
import pandas as pd
import tweepy
import time
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

Para acceder a la API de Twitter, necesitaremos generar las claves. 
Estas claves se encuentran en la configuración de la aplicación de Twitter en la pestaña Claves y tokens de acceso:

[ https://apps.twitter.com/app/new]( https://apps.twitter.com/app/new)

In [ ]:
#@title Credenciales de Twitter
TWITTER_KEY = 'gqg9kPsD4CqRBv7IzGZS8FY4m' #@param {type:"string"}
TWITTER_SECRET_KEY = 'bQA0sq1OLZoSi8UhN4qvoYtwr9ZrlJvsbBcGhWC3MHtl5UTJt1' #@param {type:"string"}

In [ ]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [ ]:
#@title Busquemos los tuits
#@markdown ### Ingresa el término de busqueda:
searchQuery = '#ElonMusk' #@param {type:"string"}
#@markdown ### Define el maximo de tweets a buscar:
#@markdown #### No descargues más de 1000.
maxTweets = 500 #@param {type:"slider", min:0, max:4500, step:100}
Filter_Retweets = True #@param {type:"boolean"}
idioma = "es" #@param ["es", "en"]

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang=idioma)
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Descargados {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

clear_output()
print("Downloaded {0} tweets".format(tweetCount))

Downloaded 500 tweets


In [ ]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df.to_excel('tweetsElonES.xlsx')
tweet_df.head()

,tweet_dt,topic,id,username,name,tweet,like_count,reply_count,retweet_count,retweeted
0,2022-04-06,#elonmusk,1511687098130173960,manu73738588,José Manuel Díaz,¡¡CUANDO #ElonMusk SE COMPRE UNA PARCELA EN MARTE...ENTONCES ME PENSARE EN COMPRARLE UN COHETE 🚀...!! 🤔\n\n¡¡MIENTRAS… https://t.co/qw7XetBECD,0,0,0,False
1,2022-04-06,#elonmusk,1511686119645143043,radiografica893,Radio Gráfica,#PuntoDePartida | Nuevas tecnologías\n\n#ElonMusk adquirió la mayoría del paquete accionario de Twitter y empieza a p… https://t.co/dL0zrtP0Z4,1,0,0,False
2,2022-04-06,#elonmusk,1511680991789789187,jef_t12,Mr J,@elonmusk compra el 9% de las acciones de Twitter 🥳🥳🥳 hay esperanza #ElonMusk #ElonMuskTwitter,0,0,0,False
3,2022-04-06,#elonmusk,1511679165795291137,idimad360,IDIMAD 360,"Pues una de las funciones más solicitadas por los usuarios de Twitter, parece que por fin va a llegar a la red soci… https://t.co/1LIbPArKJ4",0,0,1,False
4,2022-04-06,#elonmusk,1511675864052035588,cronica,Diario Crónica,Twitter podría volver a lanzar una función muy útil que tuvo en sus inicios #Seee #MeAcuerdo!! #MiráCuál #Twitter… https://t.co/ksDkXRaJHj,1,0,2,False


# Ejercicio
Realiza una busqueda en twitter de un termino en español y otro en inglés, exportalos a Excel en archivos separados.

nlp/2022_Twitter_Data_Scrapper.ipynb

> Bloque con sangría



In [ ]:
#@title Busquemos los tuits
#@markdown ### Ingresa el término de busqueda:
searchQuery = '#ElonMusk' #@param {type:"string"}
#@markdown ### Define el maximo de tweets a buscar:
#@markdown #### No descargues más de 1000.
maxTweets = 400 #@param {type:"slider", min:0, max:4500, step:100}
Filter_Retweets = True #@param {type:"boolean"}
idioma = "en" #@param ["es", "en"]

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang=idioma)
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang=idioma, max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Descargados {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

clear_output()
print("Downloaded {0} tweets".format(tweetCount))

Downloaded 463 tweets


In [ ]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df.to_excel('tweetsElonEN.xlsx')
tweet_df.head()

,tweet_dt,topic,id,username,name,tweet,like_count,reply_count,retweet_count,retweeted
0,2022-04-06,#elonmusk,1511689529794695168,pullara,David Pullara,What's even more interesting than @elonmusk's investment in @Twitter is what happened immediately after we learned… https://t.co/LdIkRpS4k0,0,0,0,False
1,2022-04-06,#elonmusk,1511689478833942533,theallineed,TheAllINeed.com,#Twitter edit button\n#ElonMusk #Theallineed #TwitterBlue\nhttps://t.co/mCbsZyBOWW https://t.co/eeTAah8Sad,0,0,0,False
2,2022-04-06,#elonmusk,1511689470395011073,info_pravda,Ukraine,@elonmusk thanks for helping #Ukrainian\n\n#ElonMusk,0,0,0,False
3,2022-04-06,#elonmusk,1511689461230448646,eprotothema,Proto Thema English,#ElonMusk One-Day Gain Dwarfs Twitter’s Profits (infographic)\n https://t.co/aBMwizJi2q https://t.co/gvl4r7wO8A,0,0,0,False
4,2022-04-06,#elonmusk,1511689446529323016,ngenidevs,NGENI developers🔥,Twitter quickly names Musk to Board of Directors after he becomes Largest shareholder\nTwitter will put #ElonMusk on… https://t.co/KQOd79iBob,0,0,0,False
